In [1]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Bidirectional
from sklearn.preprocessing import MinMaxScaler


# fibonacci
def generate_fibonacci(n):
    fib = [0, 1]
    for i in range(2, n):
        fib.append(fib[i - 1] + fib[i - 2])
    return np.array(fib, dtype=np.float32).reshape(-1, 1)


n_numbers = 25
# ile patrzymy wstecz
seq_length = 5  

data = generate_fibonacci(n_numbers)

scaler = MinMaxScaler(feature_range=(0, 1))
data_scaled = scaler.fit_transform(data)

X, y = [], []
for i in range(len(data_scaled) - seq_length):
    X.append(data_scaled[i : i + seq_length])
    y.append(data_scaled[i + seq_length])

X = np.array(X)
y = np.array(y)

# BRNN
model = Sequential(
    [
        # Bidirectional LSTM 
        Bidirectional(LSTM(64, activation="relu", input_shape=(seq_length, 1))),
        Dense(32, activation="relu"),
        Dense(1),
    ]
)

model.compile(optimizer="adam", loss="mse")

# 3. Trening
print("Rozpoczynanie treningu...")
history = model.fit(X, y, epochs=200, verbose=0)
print("Trening zakończony.")

# przewidywanie n+1
last_window = data_scaled[-seq_length:].reshape(1, seq_length, 1)
prediction_scaled = model.predict(last_window)

prediction = scaler.inverse_transform(prediction_scaled)
actual_next = data[-1][0] + data[-2][0]

print("-" * 30)
print(f"Sekwencja wejściowa: {data[-seq_length:].flatten().astype(int)}")
print(f"Przewidywana liczba Fibonacciego: {prediction[0][0]:.2f}")
print(f"Rzeczywista liczba Fibonacciego: {actual_next}")


c:\Python311\Lib\site-packages\keras\src\layers\rnn\rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Rozpoczynanie treningu...
Trening zakończony.
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 241ms/step
------------------------------
Sekwencja wejściowa: [ 6765 10946 17711 28657 46368]
Przewidywana liczba Fibonacciego: 70792.12
Rzeczywista liczba Fibonacciego: 75025.0
